In [1]:
import cv2
import numpy as np
import os

In [2]:
params_dir = os.getcwd()+'/final_camera_params/'
print(params_dir)

#load camera parameters
K = np.load(params_dir+'K.npy')
dist = np.load(params_dir+'dist.npy')

# Your chessboard size
chessboard_size = (3,6)
# iterative termination criteria, maximum iterationm and epsilon
term_criteria = (cv2.TermCriteria_EPS+ cv2.TermCriteria_MAX_ITER, 30, 0.001)
# Defining the world coordinates for 3D points
objp = np.zeros((1, chessboard_size[0] * chessboard_size[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1, 2)
# 3D object for visualization
axis = np.float32([[3,0,0], [0,3,0], [0,0,-3]]).reshape(-1,3)

c:\Users\jrwvi\OneDrive\Documents\GitHub\New folder/final_camera_params/


In [3]:
def draw_axis(img, corners, imgpts) :
    corner = tuple(corners[0].ravel().astype(int))
    img = cv2.line(img, corner, tuple(imgpts[0].ravel().astype(int)), (255,0,0), 10)
    img = cv2.line(img, corner, tuple(imgpts[1].ravel().astype(int)), (0,255,0), 10)
    img = cv2.line(img, corner, tuple(imgpts[2].ravel().astype(int)), (0,0,255), 10)
    return img

In [4]:
def augmented_image(frame, im_src, pts_src, pts_dst):
    
    # Calculate Homography
    h, status = cv2.findHomography(pts_src, pts_dst)

    # Warp source image to destination based on homography
    warped_image = cv2.warpPerspective(im_src, h, (frame.shape[1],frame.shape[0]))
            
    # Prepare a mask representing region to copy from the warped image into the original frame.
    mask = np.zeros([frame.shape[0], frame.shape[1]], dtype=np.uint8)
    cv2.fillConvexPoly(mask, np.int32(pts_dst), (255, 255, 255), cv2.LINE_AA)
    im_out = cv2.add(frame, warped_image, mask=cv2.bitwise_not(mask))
    im_out = cv2.add(im_out, warped_image)
    
    return im_out

In [5]:
template = cv2.imread('images/Templates/Template-3.png')
chessboard = cv2.imread('images/chessboard.png')
vid = cv2.VideoCapture('videos/final_exam/Dataset-1/left_output-1.avi')
im_src_size = chessboard.shape[:2]
src_points = np.float32([[0,0], [im_src_size[1],0],[im_src_size[1], im_src_size[0]] ,[0, im_src_size[0]] ])

detector = cv2.SIFT_create()

FLANN_INDEX_KDITREE = 1
flannParam = dict(algorithm = FLANN_INDEX_KDITREE, tree = 5)
search_params = dict(checks = 50)
flann = cv2.FlannBasedMatcher(flannParam, search_params)

while vid.isOpened() :
    ret, frame = vid.read()
    ret, new = vid.read()
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    if ret:
        template_kpts, template_desc = detector.detectAndCompute(template, None)
        query_kpts, query_desc = detector.detectAndCompute(frame_gray, None)
        matches = flann.knnMatch(query_desc, template_desc, k=2)
        good_matches = list()
        for m,n in matches :
            if m.distance < 0.75 * n.distance :
                good_matches.append(m)
        if len(good_matches) > 7 :
            src_pts = np.float32([ template_kpts[m.trainIdx].pt for m in good_matches ]).reshape(-1,1,2)
            dst_pts = np.float32([ query_kpts[m.queryIdx].pt for m in good_matches ]).reshape(-1,1,2)
            H, status = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 1.0)
            h, w = template.shape[:2]
            template_box = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1,1,2)
            transformed_box = cv2.perspectiveTransform(template_box, H)
            cv2.polylines(frame, [np.int32(transformed_box)], True, (0,255,0), 3)

            xCenter = (transformed_box[0][0,0] + transformed_box[2][0,0]) / 2
            yCenter = (transformed_box[0][0,1] + transformed_box[2][0,1]) / 2
            
            img = frame.copy()
            augmented = augmented_image(img, chessboard, src_points, transformed_box)
            
            found, corners = cv2.findChessboardCorners(augmented, chessboard_size, None)
            if found :
                cv2.drawChessboardCorners(augmented, chessboard_size, corners, ret)
                corners2 = cv2.cornerSubPix(frame_gray, corners, (11,11), (-1,-1), term_criteria)
                ret,rvecs, tvecs = cv2.solvePnP(objp, corners2, K, dist)
                tvecsX = tvecs[0,0] * 0.01
                tvecsY = tvecs[1,0] * 0.01
                tvecsZ = tvecs[2,0] * 0.01
                text = "X: {:.2f} Y: {:.2f} Z: {:.2f}".format(tvecsX, tvecsY, tvecsZ)
                img = cv2.putText(img, text, (int(xCenter), int(yCenter)), cv2.FONT_HERSHEY_COMPLEX, 0.75, (0,0,255), 2)
                #axis_imgpts, _ = cv2.projectPoints(axis, rvecs, tvecs, K, dist)
                #img = draw_axis(frame,corners,axis_imgpts)
                cv2.imshow("Result", img)
        if cv2.waitKey(1) & 0xFF == ord('q') :
            break
    else:
        break
vid.release()
cv2.destroyAllWindows()